In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy.stats import norm
from typing import Tuple

In [2]:
"""
SPY Options Risk Reversal Strategy Analysis
Date: 4/9/2025
Strategy: Sell 95% moneyness puts, buy 105% moneyness calls
"""

'\nSPY Options Risk Reversal Strategy Analysis\nDate: 4/9/2025\nStrategy: Sell 95% moneyness puts, buy 105% moneyness calls\n'



We were considering putting the following trade on, late afternoon as of 4/9/2025:
Sell 1000 contracts of 1-month SPY 95% moneyness put and use this premium buy
contracts of 1-month SPY 105% moneyness call.
Perform the following tasks:
1. (Python) Create a scenario analysis template for the above trade, making
reasonable assumptions.

In [34]:
class OptionsPricingModel:
    """Black-Scholes-Merton model for SPY options pricing"""
    
    def __init__(self, spot_price, risk_free_rate, dividend_yield):
        self.S0 = spot_price
        self.r = risk_free_rate
        self.q = dividend_yield  # SPY dividend yield
        
    def black_scholes_price(self, K, T, sigma, option_type='call'):
        """
        Calculate Black-Scholes price for European options
        
        Parameters:
        K: Strike price
        T: Time to maturity (in years)
        sigma: Implied volatility
        option_type: 'call' or 'put'
        """
        d1 = (np.log(self.S0 / K) + (self.r - self.q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        
        if option_type == 'call':
            price = self.S0 * np.exp(-self.q * T) * norm.cdf(d1) - K * np.exp(-self.r * T) * norm.cdf(d2)
        else:  # put
            price = K * np.exp(-self.r * T) * norm.cdf(-d2) - self.S0 * np.exp(-self.q * T) * norm.cdf(-d1)
        
        return price
    
    
    def crr_binomial_price(self, K, T, sigma, option_type='call', steps=1000, american=True):
        """
        Calculate option price using Cox-Ross-Rubinstein binomial tree
        
        Parameters:
        K: Strike price
        T: Time to maturity (in years)
        sigma: Volatility
        option_type: 'call' or 'put'
        steps: Number of time steps in the binomial tree
        american: True for American options, False for European
        """
        dt = T / steps  # Time increment
        
        # CRR parameters
        u = np.exp(sigma * np.sqrt(dt))  # Up factor
        d = 1 / u  # Down factor
        p = (np.exp((self.r - self.q) * dt) - d) / (u - d)  # Risk-neutral probability
        discount = np.exp(-self.r * dt)  # Discount factor per step
        
        # Initialize asset prices at maturity
        ST = np.zeros(steps + 1)
        for i in range(steps + 1):
            ST[i] = self.S0 * (u ** (steps - i)) * (d ** i)
        
        # Initialize option values at maturity
        if option_type == 'call':
            option_values = np.maximum(ST - K, 0)
        else:  # put
            option_values = np.maximum(K - ST, 0)
        
        # Backward induction through the tree
        for step in range(steps - 1, -1, -1):
            for i in range(step + 1):
                # Current stock price at this node
                S = self.S0 * (u ** (step - i)) * (d ** i)
                
                # Continuation value (discounted expected value)
                continuation_value = discount * (p * option_values[i] + (1 - p) * option_values[i + 1])
                
                # Early exercise value
                if option_type == 'call':
                    exercise_value = max(S - K, 0)
                else:  # put
                    exercise_value = max(K - S, 0)
                
                # For American options, take the maximum
                if american:
                    option_values[i] = max(continuation_value, exercise_value)
                else:
                    option_values[i] = continuation_value
        
        return option_values[0]

In [6]:
five_percent_otm_options_path = '../data/Option/eod_option_5%_OTM.csv'
df = pd.read_csv(five_percent_otm_options_path)
call = df[df["call_put"] == "C"]
put = df[df["call_put"] == "P"]

call_iv = call["iv"].iloc[0]
put_iv  = put[ "iv"].iloc[0]
call_strike = call["price_strike"].iloc[0]
put_strike = put["price_strike"].iloc[0]
spot_price = call["underlying_price"].iloc[0]

print(f"Call IV: {call_iv}")
print(f"Put IV: {put_iv}")
print(f"Call Strike: {call_strike}")
print(f"Put Strike: {put_strike}")
print(f"Spot Price: {spot_price}")

Call IV: 0.210945
Put IV: 0.323284
Call Strike: 571.0
Put Strike: 516.0
Spot Price: 543.37


In [41]:
# === Step 1: Set up market parameters (as of 4/9/2025) ===
spot_price = spot_price  # Example: SPY trading at $580
risk_free_rate = 0.045   # 4.5% annual risk-free rate
dividend_yield = 0.013   # ~1.3% annual dividend yield for SPY
time_to_maturity = 1/12  # 1 month = 1/12 years


# === Step 2: Initialize the pricing model ===
model = OptionsPricingModel(
    spot_price=spot_price,
    risk_free_rate=risk_free_rate,
    dividend_yield=dividend_yield
)

# Put option (SELL): 95% moneyness
put_strike = put_strike
put_contracts = 1000
put_iv = put_iv

# Call option (BUY): 105% moneyness  
call_strike = call_strike
call_contracts = 1000
call_iv = call_iv

# === Step 3: Choose binomial tree settings ===
binomial_steps = 1000  # you can tweak this (100, 500, 1000, etc.)

# === Step 4: Price the options (Black–Scholes) ===
put_price_bs = model.black_scholes_price(
    K=put_strike,
    T=time_to_maturity,
    sigma=put_iv,
    option_type='put'
)

call_price_bs = model.black_scholes_price(
    K=call_strike,
    T=time_to_maturity,
    sigma=call_iv,
    option_type='call'
)

# === Step 4b: Price the options (CRR Binomial) ===
put_price_binom = model.crr_binomial_price(
    K=put_strike,
    T=time_to_maturity,
    sigma=put_iv,
    option_type='put',
    steps=binomial_steps,
    american=True
)

call_price_binom = model.crr_binomial_price(
    K=call_strike,
    T=time_to_maturity,
    sigma=call_iv,
    option_type='call',
    steps=binomial_steps,
    american=True
)

# === Step 5: Calculate trade economics (using BS prices for sizing) ===
premium_collected = put_price_bs * put_contracts * 100  # 100 shares per contract

# Recompute how many calls you can buy with the collected premium (still using BS)
call_contracts = 1000

# Premium paid for calls (BS-based)
premium_paid = call_contracts * call_price_bs * 100
net_premium = premium_collected - premium_paid

# === Output ===
print("=== TRADE SETUP ===")
print(f"Spot Price: ${spot_price:.2f}")
print(f"Risk-free rate: {risk_free_rate*100:.2f}%")
print(f"Dividend yield: {dividend_yield*100:.2f}%")
print(f"Time to maturity: {time_to_maturity:.4f} years")
print(f"Binomial steps: {binomial_steps}")

print("\n--- PUT (SELL) 95% Moneyness ---")
print(f"Strike: ${put_strike:.2f}")
print(f"Implied Vol: {put_iv*100:.2f}%")
print(f"Contracts: {put_contracts}")
print(f"  BS price per option:   ${put_price_bs:.4f}")
print(f"  Binomial price per option: ${put_price_binom:.4f}")
print(f"  Premium collected (BS): ${premium_collected:,.2f}")

print("\n--- CALL (BUY) 105% Moneyness ---")
print(f"Strike: ${call_strike:.2f}")
print(f"Implied Vol: {call_iv*100:.2f}%")
print(f"Contracts bought (BS sizing): {call_contracts}")
print(f"  BS price per option:   ${call_price_bs:.4f}")
print(f"  Binomial price per option: ${call_price_binom:.4f}")
print(f"  Premium paid (BS): ${premium_paid:,.2f}")

print("\n=== NET PREMIUM (BS-based sizing) ===")
print(f"Net Premium: ${net_premium:,.2f}")

print("\n=== MODEL CHECK ===")
print("Put:  BS vs Binomial =", round(put_price_bs, 4), "vs", round(put_price_binom, 4))
print("Call: BS vs Binomial =", round(call_price_bs, 4), "vs", round(call_price_binom, 4))


=== TRADE SETUP ===
Spot Price: $543.37
Risk-free rate: 4.50%
Dividend yield: 1.30%
Time to maturity: 0.0833 years
Binomial steps: 1000

--- PUT (SELL) 95% Moneyness ---
Strike: $516.00
Implied Vol: 32.33%
Contracts: 1000
  BS price per option:   $8.5507
  Binomial price per option: $8.5851
  Premium collected (BS): $855,074.71

--- CALL (BUY) 105% Moneyness ---
Strike: $571.00
Implied Vol: 21.09%
Contracts bought (BS sizing): 1000
  BS price per option:   $4.2805
  Binomial price per option: $4.2796
  Premium paid (BS): $428,053.08

=== NET PREMIUM (BS-based sizing) ===
Net Premium: $427,021.63

=== MODEL CHECK ===
Put:  BS vs Binomial = 8.5507 vs 8.5851
Call: BS vs Binomial = 4.2805 vs 4.2796
